# Analysis on meals based on presence information

We've already submited many inquiries about meals made outside DF while deputies were supposed to be in DF and also meals in DF while deputies were supposed to be somewhere else (like in another country on an official mission). This notebook outlines all types of analysis we can do around that with datasets we currently have around and it also makes use of more recent versions of datasets we've collected over the past couple months.

In [1]:
import re
import time
from datetime import timedelta

import pandas as pd
import numpy as np

from serenata_toolbox.datasets import fetch

In [2]:
from IPython.display import HTML

def report(df, cols):
    df = df.copy()
    df['receipt'] = df.apply(link_to_receipt, axis=1)
    df['document_id'] = df.apply(link_to_jarbas, axis=1)
    return HTML(df[cols].to_html(escape=False))

def link_to_jarbas(r):
    return '<a target="_blank" href="http://jarbas.serenatadeamor.org/#/document_id/{0}">{0}</a>'.format(r.document_id)

DOCUMENT_URL = (
    'http://www.camara.gov.br/'
    'cota-parlamentar/documentos/publ/{}/{}/{}.pdf'
)
def link_to_receipt(r):
    url = DOCUMENT_URL.format(r.applicant_id, r.year, r.document_id)
    return '<a target="_blank" href="{0}">RECEIPT</a>'.format(url)

pd.set_option('display.max_colwidth', 1500)

## Data preparation

In [6]:
# fetch("2017-02-15-receipts-texts.xz", "../data")
# fetch("2017-03-15-reimbursements.xz", "../data")
fetch("2017-05-21-companies-no-geolocation.xz", "../data")

# TODO: Need to make sure the datasets have been uploaded to the S3 bucket
# fetch("2017-04-19-official-missions.xz", "../data")
# fetch("2017-05-29-session-start-times.xz", "../data")
# fetch("2017-05-29-presences.xz", "../data")
# fetch("2017-05-29-deputies.xz", "../data")
# fetch("2017-05-29-speeches.xz", "../data")

In [7]:
reimbursements = pd.read_csv('../data/2017-03-15-reimbursements.xz', dtype={'cnpj_cpf': np.str}, low_memory=False)
print("Total reimbursements:", len(reimbursements))

# Reduce dataset to current term
reimbursements = reimbursements.query('year >= 2015')
reimbursements['issue_date'] = pd.to_datetime(reimbursements['issue_date'], format="%Y-%m-%dT%H:%M:%S").dt.date
print("Reimbursements in this term:", len(reimbursements))

# Reduce dataset to meals
meals = reimbursements.query('subquota_description == "Congressperson meal"')
print("Meals in this term:", len(meals))

Total reimbursements: 1580942
Reimbursements in this term: 428002
Meals in this term: 61757


In [8]:
companies = pd.read_csv('../data/2017-05-21-companies-no-geolocation.xz', low_memory=False)
companies['cnpj'] = companies['cnpj'].str.replace(r'[\.\/\-]', '')

# Reduce dataset to meals with matching company and state info
meals = pd.merge(
    meals, 
    companies[['cnpj', 'city', 'state', 'main_activity', 'name']], 
    left_on='cnpj_cpf', 
    right_on='cnpj',
    suffixes=('_congressperson', '_company')
)
print("Meals with matching companies:", len(meals))

meals = meals[~meals.state_company.isnull()]
print("Meals with known company state:", len(meals))
meals = meals.rename(index=str, columns={
    "state_congressperson": "congressperson_state",
    "name": "company_name", 
    "city": "company_city", 
    "state_company": "company_state"
})

Meals with matching companies: 61428
Meals with known company state: 60620


In [9]:
texts = pd.read_csv('../data/2017-02-15-receipts-texts.xz', dtype={'text': np.str}, low_memory=False)
texts['text'] = texts.text.str.upper()

meals_with_ts = texts.merge(meals[['document_id']], on='document_id')
print("Meals with OCR:", len(meals_with_ts))

Meals with OCR: 55615


In [10]:
def extract_timestamps(text):
    return re.findall('[0-9][0-9]:[0-9][0-9]:[0-9][0-9]', str(text))
meals_with_ts['timestamps'] = meals_with_ts.text.apply(extract_timestamps)

def parse_timestamps(ts):
    try:
        return time.strptime(ts, "%H:%M:%S")
    except:
        return None
meals_with_ts['timestamps'] = meals_with_ts.timestamps.apply(lambda ts: list(map(parse_timestamps, ts)))
meals_with_ts = meals_with_ts[meals_with_ts.apply(lambda r: len(r.timestamps) > 0, axis=1)]

meals = pd.merge(
    left=meals,
    right=meals_with_ts[['document_id', 'timestamps']],
    how='left',
    on='document_id'
)
print("Meals with timestamps:", len(meals[~meals.timestamps.isnull()]))

Meals with timestamps: 31284


In [11]:
meals = meals[[
    'document_id',
    'applicant_id',
    'congressperson_document',
    'congressperson_id',
    'congressperson_name',
    'congressperson_state',
    'party',
    'year',
    'issue_date',
    'total_net_value',
    'company_name',
    'company_state',
    'timestamps'
]]

## Meals outside DF while the congressperson was present in a session

There are at least two ways we can identify that:
* Making an approximation of the total time the congressperson was in Brasilia (like more than X hours) and checking for reimbursements on days they were there for a long period of time
* Leveraging OCR data from receipts and grabbing their timestamps

In [12]:
deputies = pd.read_csv('../data/2017-04-19-deputies.xz', low_memory=False)
print("Total deputies:", len(deputies))

sessions = pd.read_csv('../data/2017-04-19-session-start-times.xz', low_memory=False)
sessions['date'] = pd.to_datetime(sessions['date'], format="%Y-%m-%dT%H:%M:%S").dt.date
sessions['started_at'] = pd.to_datetime(sessions['started_at'], format="%Y-%m-%dT%H:%M:%S")
print("Session records:", len(sessions))

presences = pd.read_csv('../data/2017-04-19-presences.xz', low_memory=False)
presences['date'] = pd.to_datetime(presences['date'], format="%Y-%m-%dT%H:%M:%S").dt.date
presences.sort_values('date', ascending=False)
print("Presence records:", len(presences))

presences = presences.query('presence == "Present"')
print("Presence records with presence confirmed:", len(presences))

# Match presence with session and deputy info
presences = pd.merge(presences, sessions, on=['date', 'session'])
print("Presence records with session matched:", len(presences))

presences = pd.merge(deputies, presences, on='congressperson_document')
print("Presence records with deputies matched:", len(presences))

Total deputies: 513
Session records: 450
Presence records: 212863
Presence records with presence confirmed: 179280
Presence records with session matched: 179280
Presence records with deputies matched: 179280


In [13]:
presences['first_session_at'] = presences['started_at']
presences['last_session_at'] = presences['started_at']
presences['total_sessions'] = 0
aggregations = {
    'first_session_at': 'min',
    'last_session_at': 'max',
    'total_sessions': 'count',
    'session': lambda x: "{%s}" % ', '.join(x.astype(str)),
}
presences = presences.groupby(['congressperson_id', 'date'], as_index=False).agg(aggregations)
print("Confirmed presences grouped by date and congressperson:", len(presences))

Confirmed presences grouped by date and congressperson: 104582


In [17]:
meals_outside_df = meals.query('company_state != "DF"')
meals_outside_df_while_in_df = pd.merge(
    left=presences, 
    right=meals_outside_df, 
    left_on=['congressperson_id', 'date'],
    right_on=['congressperson_id', 'issue_date']
)
print("Meals outside DF on days that the congressperson was there:", len(meals_outside_df_while_in_df))

meals_outside_df_while_in_df = meals_outside_df_while_in_df.query('company_name != "GOL LINHAS AEREAS S.A."')
print("Meals excluding expenses on flight:", len(meals_outside_df_while_in_df))

meals_outside_df_while_in_df = meals_outside_df_while_in_df[
    ~meals_outside_df_while_in_df.company_name.str.contains('HOTEL')
]
# We keep hotels out of the equation for this analysis since it they usually
# have meals made on multiple days. Not to say that the bill might dated from
# the day that the congressperson was back.
print("Meals excluding expenses on hotels:", len(meals_outside_df_while_in_df))

Meals outside DF on days that the congressperson was there: 2898
Meals excluding expenses on flight: 2841
Meals excluding expenses on hotels: 2784


In [18]:
def score(meal):
    return score_by_time(meal) + score_by_ts(meal)

def score_by_time(meal):
    if meal.total_sessions > 1:
        return meal.last_session_at.hour - meal.first_session_at.hour + (meal.total_sessions - 1)
    else:
        return 0
    
def score_by_ts(meal):
    if meal.timestamps == '':
        return 0
    
    occurences = 0
    for ts in meal.timestamps:
        if ts == None:
            continue
        if (meal.first_session_at.hour-1) < ts.tm_hour < (meal.last_session_at.hour+1):
            occurences += 1
    return occurences * 10
    
suspects = meals_outside_df_while_in_df.copy()
suspects.timestamps.fillna('', inplace=True)
suspects['score'] = suspects.apply(score, axis='columns')
suspects = suspects.query('score > 0').sort_values('score', ascending=False)
print("Suspicious reimbursements:", len(suspects))

In [24]:
report(suspects.head(50), [
    'document_id',
    'receipt',
    'issue_date', 
    'congressperson_name',
    'company_name',
    'company_state', 
    'total_net_value',
    'score',
    'first_session_at', 
    'last_session_at',
])

,document_id,receipt,issue_date,congressperson_name,company_name,company_state,total_net_value,score,first_session_at,last_session_at
2877,6136366,RECEIPT,2016-10-24,FRANKLIN LIMA,SOL PANAMBY AGROEMPRESARIAL LTDA.,SP,11.50,60,2016-10-24 17:26:11,2016-10-24 17:26:11
365,6099888,RECEIPT,2016-09-12,LOBBE NETO,RITA DE CASSIA C. CABRERA LANCHONETE - EPP,SP,15.00,57,2016-09-12 13:00:02,2016-09-12 19:00:20
216,5941318,RECEIPT,2016-03-15,CELSO JACOB,TEIXEIRA BATISTA BUFES LTDA - EPP,RJ,165.00,56,2016-03-15 13:55:10,2016-03-15 18:13:27
1891,6155695,RECEIPT,2016-11-23,MARCOS ROGÉRIO,CHURRASCARIA LOCATELLI LTDA - ME,RO,67.99,54,2016-11-23 09:23:49,2016-11-23 21:30:17
1131,5735507,RECEIPT,2015-06-17,VANDERLEI MACRIS,CAPISCE COMERCIO DE ALIMENTOS LTDA - ME,SP,28.30,34,2015-06-17 09:00:15,2015-06-17 21:39:40
1718,6048825,RECEIPT,2016-04-27,KEIKO OTA,BAR E RESTAURANTE BOINABRASA LTDA - EPP,SP,34.60,34,2016-04-27 09:29:11,2016-04-27 20:56:20
1892,6155736,RECEIPT,2016-11-23,MARCOS ROGÉRIO,CHURRASCARIA LOCATELLI LTDA - ME,RO,53.99,34,2016-11-23 09:23:49,2016-11-23 21:30:17
659,5853739,RECEIPT,2015-11-11,ADEMIR CAMILO,MULTI FORMATO DISTRIBUIDORA SOCIEDADE ANONIMA,MG,36.80,32,2015-11-11 13:00:07,2015-11-11 23:26:05
549,6092898,RECEIPT,2016-08-30,SARAIVA FELIPE,CABANA DA MANTIQUEIRA TURISMO LTDA - EPP,MG,40.23,32,2016-08-30 09:27:42,2016-08-30 19:14:23
1248,5705076,RECEIPT,2015-05-26,MARCO TEBALDI,CHURRASCARIA RECH LTDA - ME,SC,20.00,31,2015-05-26 12:00:12,2015-05-26 21:01:53


## Meals outside DF while the congressperson was giving a speech

Like presence in sessions, we can identify suspicious reimbursements based on:
* Making an approximation of the total time the congressperson was in Brasilia (like more than X hours) and checking for reimbursements on days they were there for a long period of time
* Leveraging OCR data from receipts and grabbing their timestamps

In [25]:
speeches = pd.read_csv('../data/2017-04-21-speeches.xz', low_memory=False)
print("Total speeches:", len(speeches))
# Clean up the data, see https://gist.github.com/fgrehm/bb0e1f6fef55082074d9a0258cf45391 for background
speeches = speeches[~speeches['speech_speaker_party'].isnull()]
speeches['speech_speaker_name'] = speeches['speech_speaker_name'].str.replace('\s+\(PRESIDENTE\)', '')
speeches['speech_speaker_party'] = speeches['speech_speaker_party'].str.upper()
speeches['speech_speaker_party'] = speeches['speech_speaker_party'].str.replace('PDSB', 'PSDB')
speeches['session_date'] = pd.to_datetime(speeches['session_date'], format="%Y-%m-%dT%H:%M:%S").dt.date
speeches['speech_started_at'] = pd.to_datetime(speeches['speech_started_at'], format="%Y-%m-%dT%H:%M:%S")
speeches = speeches[[
    'session_date',
    'speech_speaker_name', 
    'speech_speaker_party',
    'speech_speaker_state',
    'speech_started_at',
]]
print("Speeches by politicians:", len(speeches))

Total speeches: 55089
Speeches by politicians: 53625


In [26]:
speeches = pd.merge(
            speeches, 
            deputies, 
            left_on=['speech_speaker_name', 'speech_speaker_party', 'speech_speaker_state'],
            right_on=['congressperson_name', 'party', 'state'])
print("Speeches with matching deputy:", len(speeches))

Speeches with matching deputy: 41172


In [27]:
speeches['first_speech_at'] = speeches['speech_started_at']
speeches['last_speech_at'] = speeches['speech_started_at']
speeches['total_speeches'] = 0
aggregations = {
    'last_speech_at': 'max',
    'first_speech_at': 'min',
    'total_speeches': 'count',
}
speeches = speeches.groupby(['congressperson_document', 'session_date'], as_index=False).agg(aggregations)
print("Speeches grouped by date:", len(speeches))

Speeches grouped by date: 24404


In [29]:
meals_outside_df_during_speeches = pd.merge(
    left=speeches, 
    right=meals_outside_df, 
    left_on=['congressperson_document', 'session_date'],
    right_on=['congressperson_document', 'issue_date']
)
print("Meals outside DF on days that the congressperson gave a speech:", len(meals_outside_df_during_speeches))

meals_outside_df_during_speeches = meals_outside_df_during_speeches.query('company_name != "GOL LINHAS AEREAS S.A."')
print("Meals excluding expenses on flight:", len(meals_outside_df_during_speeches))

meals_outside_df_during_speeches = meals_outside_df_during_speeches[
    ~meals_outside_df_during_speeches.company_name.str.contains('HOTEL')
]
print("Meals excluding expenses on hotels:", len(meals_outside_df_during_speeches))

Meals outside DF on days that the congressperson gave a speech: 614
Meals excluding expenses on flight: 596
Meals excluding expenses on hotels: 590


In [34]:
def score(meal):
    return score_by_time(meal) + score_by_ts(meal)

def score_by_time(meal):
    if meal.total_speeches > 1:
        return meal.first_speech_at.hour - meal.last_speech_at.hour + (meal.total_speeches - 1)
    else:
        return 0
    
def score_by_ts(meal):
    if meal.timestamps == '':
        return 0
    
    occurences = 0
    for ts in meal.timestamps:
        if ts == None:
            continue
        if (meal.first_speech_at.hour-1) < ts.tm_hour < (meal.last_speech_at.hour+1):
            occurences += 1
    return occurences * 10
    
suspects2 = meals_outside_df_during_speeches.copy()
suspects2.timestamps.fillna('', inplace=True)
suspects2['score'] = suspects2.apply(score, axis='columns')
suspects2 = suspects2.query('score > 0').sort_values('score', ascending=False)
print("Suspicious reimbursements:", len(suspects2))

suspects2 = suspects2[~suspects2.document_id.isin(suspects.document_id)]
print("Suspicious reimbursements that were not found using presences info:", len(suspects2))

Suspicious reimbursements: 104
Suspicious reimbursements that were not found using presences info: 53


In [35]:
report(suspects2.head(20), [
    'document_id',
    'receipt',
    'issue_date', 
    'congressperson_name',
    'score',
    'company_state', 
    'total_net_value', 
    'first_speech_at', 
    'last_speech_at',
    'company_name'
])

,document_id,receipt,issue_date,congressperson_name,score,company_state,total_net_value,first_speech_at,last_speech_at,company_name
163,5864392,RECEIPT,2015-11-19,RENZO BRAZ,20,RJ,38.19,2015-11-19 10:44:00,2015-11-19 10:44:00,BAR E RESTAURANTE SERRA DO CAPIM LTDA - EPP
420,5764399,RECEIPT,2015-08-06,MARCO TEBALDI,20,SC,42.10,2015-08-06 12:40:00,2015-08-06 12:40:00,4 S EMPREENDIMENTOS LTDA - ME
487,5866763,RECEIPT,2015-12-02,AFONSO MOTTA,17,RS,30.35,2015-12-02 09:00:00,2015-12-02 13:52:00,SHALON LANCHERIA - EIRELI - EPP
383,5673697,RECEIPT,2015-04-23,CELSO MALDANER,14,SC,35.50,2015-04-23 09:12:00,2015-04-23 17:54:00,CHURRASCARIA RIO SUL LTDA - ME
259,5682390,RECEIPT,2015-05-08,GOULART,11,SP,77.00,2015-05-08 15:00:00,2015-05-08 15:00:00,ALMANARA RESTAURANTES E LANCHONETES LTDA
506,5823950,RECEIPT,2015-10-15,BOHN GASS,10,SP,22.50,2015-10-15 12:36:00,2015-10-15 12:36:00,GR SERVICOS E ALIMENTACAO LTDA.
356,5635756,RECEIPT,2015-03-17,RUBENS BUENO,10,PR,2.50,2015-03-17 17:22:00,2015-03-17 21:12:00,RITA DE FATIMA DE CASTRO - ME
470,5681292,RECEIPT,2015-04-27,VALDIR COLATTO,7,SC,34.00,2015-04-27 19:26:00,2015-04-27 19:26:00,RESTAURANTE ZAMPRONIO & DE FILTRO LTDA - ME
565,5798316,RECEIPT,2015-08-13,POMPEO DE MATTOS,4,RS,99.00,2015-08-13 12:50:00,2015-08-13 12:50:00,BATISTA FONTANA - EPP
363,5959367,RECEIPT,2016-03-31,RUBENS BUENO,3,PR,13.00,2016-03-31 14:32:00,2016-03-31 14:32:00,RESTAURANTE BIG MEAL LTDA - ME


## Meals in DF while the congressperson was outside DF

In the past we also found cases where the congressperson was on an official mission outside DF but requested a reimbursement for a meal in DF

In [36]:
missions = pd.read_csv('../data/2017-04-19-official-missions.xz', low_memory=False)
missions.participant = missions.participant.apply(lambda x: x.upper())
missions.start = pd.to_datetime(missions.start, format="%Y-%m-%d").dt.date
missions.end = pd.to_datetime(missions.end, format="%Y-%m-%d").dt.date

print("Total official mission records:", len(missions))

missions = missions.merge(
    deputies, 
    left_on=['participant'],
    right_on=['congressperson_name']
)
print("Total official mission records with deputies:", len(missions))

Total official mission records: 1033
Total official mission records with deputies: 934


In [38]:
meals_in_df = meals.query('company_state == "DF"')
print("Meals in DF:", len(meals_in_df))

Meals in DF: 37802


In [39]:
suspects3 = []
for idx, mission in missions.iterrows():
    meals_while_in_mission = meals_in_df[
        (meals_in_df.issue_date >= (mission.start + timedelta(days=1))) \
        & (meals_in_df.issue_date <= (mission.end - timedelta(days=1))) \
        & (meals_in_df.congressperson_id == mission.congressperson_id)
    ]
    if (len(meals_while_in_mission) == 0):
        continue
    for _, m in meals_while_in_mission.iterrows():
        suspects3.append([
            m.document_id,
            m.applicant_id,
            m.year,
            m.issue_date,
            m.congressperson_name,
            m.total_net_value,
            m.company_state,
            m.company_name,
            "{}<br>{}<br><i>From '{}' to '{}'</i>".format(
                mission.destination,
                mission.subject,
                mission.start,
                mission.end
            )
        ])    
    
suspects3 = pd.DataFrame(suspects3, columns=[
    'document_id',
    'applicant_id',
    'year',
    'issue_date', 
    'congressperson_name',
    'total_net_value',
    'company_state', 
    'company_name',
    'mission'
])

In [40]:
report(suspects3, [
    'document_id',
    'receipt',
    'issue_date', 
    'congressperson_name',
    'total_net_value', 
    'mission',
    'company_name',
    'company_state'
])

,document_id,receipt,issue_date,congressperson_name,total_net_value,mission,company_name,company_state
0,5820753,RECEIPT,2015-09-16,HIRAN GONÇALVES,74.58,"Barcelona, LondresRealizar visitas técnicas às cidades de Barcelona/Espanha e Londres/Reino Unido, a fim de avaliar os legados esportivos dos Jogos Olímpicos de 1992 e de 2012.From '2015-09-12' to '2015-09-19'",DUDUSAN RESTAURANTE E FAST FOOD LTDA - EPP,DF
1,5817751,RECEIPT,2015-10-07,ELCIONE BARBALHO,20.50,"Cidade do MéxicoParticipar da WIP Mexico Summit 2015, a ser promovida pelo Women in Parliaments Global Forum.From '2015-10-06' to '2015-10-10'",SERVICO NACIONAL DE APRENDIZAGEM COMERCIAL SENAC,DF
2,5817712,RECEIPT,2015-10-08,ELCIONE BARBALHO,50.69,"Cidade do MéxicoParticipar da WIP Mexico Summit 2015, a ser promovida pelo Women in Parliaments Global Forum.From '2015-10-06' to '2015-10-10'",GOODS RESTAURANTE E LANCHONETE EIRELI - ME,DF
3,6174589,RECEIPT,2016-12-08,ELCIONE BARBALHO,25.04,"Fortaleza/CEMesa Redonda na Assembleia Legislativa do Estado do Ceará, em Fortaleza, no dia 08/12/2016, das 9 às 18 horas.From '2016-12-07' to '2016-12-09'",GOODS RESTAURANTE E LANCHONETE EIRELI - ME,DF
4,5781706,RECEIPT,2015-09-01,ELIZIANE GAMA,54.53,"Curitiba/PRRealização de diligências e oitivas em Curitiba, PR, com vistas a tomar o depoimento de algumas pessoas, as quais se encontram detidas em decorrência da Operação Lava-Jato, no período compreendido entre os dias 31/08 e 03/09/2015.From '2015-08-31' to '2015-09-03'",SERVICO NACIONAL DE APRENDIZAGEM COMERCIAL SENAC,DF
5,5707683,RECEIPT,2015-05-18,DARCÍSIO PERONDI,138.00,"Ipojuca/PERealização de diligência com visita às obras da Refinaria Abreu e Lima, localizada no Município de Ipojuca, PE, no dia 18/5/2015.From '2015-05-17' to '2015-05-19'",WINE COMPANY BEBIDAS LTDA - EPP,DF
6,5664780,RECEIPT,2015-04-23,SÓSTENES CAVALCANTE,20.00,Buenos AiresConferência Global Parlamentar e Fé.From '2015-04-20' to '2015-04-25',SERVICO NACIONAL DE APRENDIZAGEM COMERCIAL SENAC,DF
7,5653496,RECEIPT,2015-04-13,FLAVINHO,29.80,Las VegasNAB SHOW 2015 Las Vegas - EUAFrom '2015-04-12' to '2015-04-17',JLM RESTAURANTE LTDA - EPP,DF
8,5872138,RECEIPT,2015-12-08,RÔMULO GOUVEIA,2.90,"Nova IorqueParticipar, como observador, de debates e reuniões da 70ª sessão da Assembleia-Geral da Organização das Nações Unidas (ONU). O deputado utilizará a cota para upgrade para classe executiva.From '2015-11-09' to '2015-12-14'",SERVICO NACIONAL DE APRENDIZAGEM COMERCIAL SENAC,DF
9,5872515,RECEIPT,2015-12-10,RÔMULO GOUVEIA,69.80,"Nova IorqueParticipar, como observador, de debates e reuniões da 70ª sessão da Assembleia-Geral da Organização das Nações Unidas (ONU). O deputado utilizará a cota para upgrade para classe executiva.From '2015-11-09' to '2015-12-14'",SERVICO NACIONAL DE APRENDIZAGEM COMERCIAL SENAC,DF
